In [11]:
import json
from warnings import filterwarnings as _fw_

import numpy as np
import pandas as pd
from numpy import median, isnan

_fw_('ignore')

In [12]:
df = pd.read_csv('df.csv', encoding='utf-8').drop(columns='Unnamed: 0')
# df

In [13]:
with open('binary_columns.json', 'r', encoding='utf-8') as file:
    binary_columns = json.load(file)
# binary_columns[:-2]

In [14]:
target = binary_columns[0:13]
school_head5 = binary_columns[14:30]
school_tail5 = binary_columns[31:46]
course = binary_columns[47:56]
profession = binary_columns[57:66]

# АНАЛИЗ ПОЛОВОЗРАСТНОЙ ГРУППЫ

In [15]:
mask_E = {'Предпочитаю не отвечать': "-1 >> Предпочитаю не отвечать",
          'Нет дохода': "0 >> Нет дохода",
          'Менее 10 000 рублей': "1 >> Менее 10 000 рублей",
          '10 000 – 20 000 рублей': "2 >> 10 000 – 20 000 рублей",
          '20 001 – 30 000 рублей': "3 >> 20 001 – 30 000 рублей",
          '30 001 – 40 000 рублей': "4 >> 30 001 – 40 000 рублей",
          '40 001 – 50 000 рублей': "5 >> 40 001 – 50 000 рублей",
          '50 001 – 70 000 рублей': "6 >> 50 001 – 70 000 рублей",
          '70 001 – 100 000 рублей': "7 >> 70 001 – 100 000 рублей",
          '100 001 – 150 000 рублей': "8 >> 100 001 – 150 000 рублей",
          'Более 150 000 рублей': "9 >> Более 150 000 рублей", }

mask_A = {10: '10+',
          20: '20+',
          30: '30+',
          40: '40+',
          50: '50+',
          60: '60+',
          70: '70+',
          80: '80+',
          90: '90+',
          }

In [16]:
df_SAE = df.copy()
df_SAE['_EARNINGS'] = df_SAE['_EARNINGS'].map(mask_E)

In [17]:
# df_SAE

In [18]:
df_SAE['_AGE'] = df_SAE['_AGE'].fillna(int(median(df_SAE['_AGE'][~isnan(df_SAE['_AGE'])])))
df_SAE['AGE_CATEGORY'] = (df_SAE['_AGE'] // 10).apply(int) * 10
df_SAE['AGE_CATEGORY'] = df_SAE['AGE_CATEGORY'].map(mask_A)
to_pivot = df_SAE[['RESPONSE_ID',
                   'SALES_CHANNEL',
                   '_COURSE Насколько вы определились с выбором профессии, которую хотите изучать?',
                   'COURSE Аналитик данных',
                   'COURSE Инженер по тестированию',
                   'COURSE Python-разработчик',
                   'COURSE Java-разработчик',
                   'COURSE Android-разработчик',
                   'COURSE Frontend-разработчик',
                   'COURSE Маркетинг',
                   'COURSE Веб-дизайн',
                   'SEX',
                   'AGE_CATEGORY',
                   '_EARNINGS']]
to_pivot = to_pivot[~to_pivot['_COURSE Насколько вы определились с выбором профессии, которую хотите изучать?'].isin(['Затрудняюсь ответить'])]
to_pivot

,RESPONSE_ID,SALES_CHANNEL,"_COURSE Насколько вы определились с выбором профессии, которую хотите изучать?",COURSE Аналитик данных,COURSE Инженер по тестированию,COURSE Python-разработчик,COURSE Java-разработчик,COURSE Android-разработчик,COURSE Frontend-разработчик,COURSE Маркетинг,COURSE Веб-дизайн,SEX,AGE_CATEGORY,_EARNINGS
0,4,От блогера на YouTube (укажите имя блогера),Выбираю между несколькими профессиями,0,0,1,0,0,0,0,0,1,20+,8 >> 100 001 – 150 000 рублей
1,5,От коллег на работе,Не определился(-ась) с выбором,0,1,1,1,0,0,0,0,1,20+,6 >> 50 001 – 70 000 рублей
2,7,"От знакомых, друзей",Выбираю между несколькими профессиями,1,1,1,1,0,0,0,0,1,20+,4 >> 30 001 – 40 000 рублей
3,10,Через поиск в Google / Яндекс / другой поисков...,"Точно знаю, какую профессию хочу изучить",1,0,0,0,0,0,0,0,0,30+,7 >> 70 001 – 100 000 рублей
4,11,От блогера на YouTube (укажите имя блогера),Не определился(-ась) с выбором,0,0,0,0,1,0,0,0,1,30+,-1 >> Предпочитаю не отвечать
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9754,12117,"Реклама в Facebook, Вконтакте",Не определился(-ась) с выбором,1,0,0,0,0,0,0,1,0,30+,2 >> 10 000 – 20 000 рублей
9755,12118,Другое,Не определился(-ась) с выбором,1,0,0,0,0,0,0,1,0,20+,4 >> 30 001 – 40 000 рублей
9756,12120,Пост в Telegram,Выбираю между несколькими профессиями,1,0,0,0,0,0,1,1,0,20+,5 >> 40 001 – 50 000 рублей
9757,12122,"Комментарии, отзывы в соцсетях",Не определился(-ась) с выбором,1,0,0,0,0,0,1,0,0,40+,7 >> 70 001 – 100 000 рублей


In [19]:
table = pd.pivot_table(to_pivot,
                       values=course[:-1],
                       index='AGE_CATEGORY',
                       columns='_COURSE Насколько вы определились с выбором профессии, которую хотите изучать?',
                       aggfunc=np.mean,
                       fill_value=0)
table

COURSE Android-разработчик  \
_COURSE Насколько вы определились с выбором профессии, которую хотите изучать? Выбираю между несколькими профессиями   
AGE_CATEGORY                                                                                                           
10+                                                                                                         0.161616   
20+                                                                                                         0.156966   
30+                                                                                                         0.185996   
40+                                                                                                         0.157609   
50+                                                                                                         0.139535   
60+                                                                                                         0.171429   
70+                                                                                                         0.294118   
80+                                                                                                         0.000000   

                                                                                                               \
_COURSE Насколько вы определились с выбором профессии, которую хотите изучать? Не определился(-ась) с выбором   
AGE_CATEGORY                                                                                                    
10+                                                                                                  0.115942   
20+                                                                                                  0.135955   
30+                                                                                                  0.157525   
40+                                                                                                  0.130000   
50+                                                                                                  0.103093   
60+                                                                                                  0.272727   
70+                                                                                                  0.000000   
80+                                                                                                  0.000000   

                                                                                                                         \
_COURSE Насколько вы определились с выбором профессии, которую хотите изучать? Точно знаю, какую профессию хочу изучить   
AGE_CATEGORY                                                                                                              
10+                                                                                                            0.128492   
20+                                                                                                            0.112150   
30+                                                                                                            0.084392   
40+                                                                                                            0.083951   
50+                                                                                                            0.080645   
60+                                                                                                            0.058824   
70+                                                                                                            0.000000   
80+                                                                                                            0.000000   

                                                                                         COURSE Frontend-разработчик  \
_COURSE Насколько вы определились с выбором професс